In [1]:
!condapip install surprise

  Running setup.py bdist_wheel for scikit-surprise: started
  Running setup.py bdist_wheel for scikit-surprise: finished with status 'done'
  Stored in directory: C:\Users\J\AppData\Local\pip\Cache\wheels\72\f4\52\a74e2689f98bed7029e2b3da4ade72024ba0f1975a990e283c
Successfully built scikit-surprise


In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf

In [2]:
# Load the movielens-100k dataset (download it if needed),
# and split it into 3 folds for cross-validation.
data = Dataset.load_builtin('ml-100k')
data.split(n_folds=3)


In [3]:
# We'll use the famous SVD algorithm.
algo = SVD()

In [4]:
# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9433
MAE:  0.7442
------------
Fold 2
RMSE: 0.9396
MAE:  0.7403
------------
Fold 3
RMSE: 0.9505
MAE:  0.7510
------------
------------
Mean RMSE: 0.9445
Mean MAE : 0.7451
------------
------------


In [5]:
print_perf(perf)

        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.9433  0.9396  0.9505  0.9445  
MAE     0.7442  0.7403  0.7510  0.7451  


In [11]:
import os
file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')

In [12]:
file_path

'C:\\Users\\J/.surprise_data/ml-100k/ml-100k/u.data'

In [14]:
from surprise import Reader
reader = Reader(line_format='user item rating timestamp', sep='\t')

In [16]:
data = Dataset.load_from_file(file_path, reader=reader)

In [18]:
data.split(n_folds=5)  # data can now be used normally

In [21]:
ratings_dict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 'user_foo'],
                'rating': [3, 2, 4, 3, 1]}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
data.split(2)  # data can now be used normally

In [24]:
# path to dataset folder
files_dir = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/')

# This time, we'll use the built-in reader.
reader = Reader('ml-100k')

# folds_files is a list of tuples containing file paths:
# [(u1.base, u1.test), (u2.base, u2.test), ... (u5.base, u5.test)]
train_file = files_dir + 'u%d.base'
test_file = files_dir + 'u%d.test'
folds_files = [(train_file % i, test_file % i) for i in (1, 2, 3, 4, 5)]

data = Dataset.load_from_folds(folds_files, reader=reader)

In [30]:
from surprise import GridSearch
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
grid_search = GridSearch(SVD, param_grid, measures=['RMSE', 'FCP'])

[{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}]


In [34]:
data = Dataset.load_builtin('ml-100k')
data.split(n_folds=3)

grid_search.evaluate(data)

------------
Parameters combination 1 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
------------
Mean RMSE: 0.9966
Mean FCP : 0.6849
------------
------------
Parameters combination 2 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}
------------
Mean RMSE: 1.0032
Mean FCP : 0.6875
------------
------------
Parameters combination 3 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}
------------
Mean RMSE: 0.9734
Mean FCP : 0.6949
------------
------------
Parameters combination 4 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}
------------
Mean RMSE: 0.9821
Mean FCP : 0.6950
------------
------------
Parameters combination 5 of 8
params:  {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}
------------
Mean RMSE: 0.9778
Mean FCP : 0.6933
------------
------------
Parameters combination 6 of 8
params:  {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}
------------
Mean RMSE: 0.9860
Mean FCP : 0.6942
------------
------------
Paramet

In [35]:
# best RMSE score
print(grid_search.best_score['RMSE'])
# >>> 0.96117566386

# combination of parameters that gave the best RMSE score
print(grid_search.best_params['RMSE'])
# >>> {'reg_all': 0.4, 'lr_all': 0.005, 'n_epochs': 10}

# best FCP score
print(grid_search.best_score['FCP'])
# >>> 0.702279736531

# combination of parameters that gave the best FCP score
print(grid_search.best_params['FCP'])
# >>> {'reg_all': 0.6, 'lr_all': 0.005, 'n_epochs': 10}

0.963493436177
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
0.698941629374
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}


In [36]:
import pandas as pd  # noqa

results_df = pd.DataFrame.from_dict(grid_search.cv_results)
print(results_df)

        FCP      RMSE  lr_all  n_epochs  \
0  0.684919  0.996650   0.002         5   
1  0.687476  1.003187   0.002         5   
2  0.694875  0.973352   0.005         5   
3  0.694951  0.982132   0.005         5   
4  0.693285  0.977786   0.002        10   
5  0.694190  0.985967   0.002        10   
6  0.698788  0.963493   0.005        10   
7  0.698942  0.973373   0.005        10   

                                              params  \
0   {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}   
1   {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}   
2   {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}   
3   {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}   
4  {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}   
5  {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}   
6  {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}   
7  {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}   

                                            scores  
0  {'RMSE': 0.996649891378, 'FCP': 0.684919065642}  
1

In [37]:
results_df

,FCP,RMSE,lr_all,n_epochs,params,scores
0,0.684919,0.996650,0.002,5,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}","{'RMSE': 0.996649891378, 'FCP': 0.684919065642}"
1,0.687476,1.003187,0.002,5,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}","{'RMSE': 1.00318662473, 'FCP': 0.687475792726}"
2,0.694875,0.973352,0.005,5,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}","{'RMSE': 0.973352194922, 'FCP': 0.694875353141}"
3,0.694951,0.982132,0.005,5,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}","{'RMSE': 0.982131699812, 'FCP': 0.69495120474}"
4,0.693285,0.977786,0.002,10,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}","{'RMSE': 0.977786144911, 'FCP': 0.693285197112}"
5,0.694190,0.985967,0.002,10,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}","{'RMSE': 0.985966657639, 'FCP': 0.694189921092}"
6,0.698788,0.963493,0.005,10,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}","{'RMSE': 0.963493436177, 'FCP': 0.698787698298}"
7,0.698942,0.973373,0.005,10,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}","{'RMSE': 0.973372981761, 'FCP': 0.698941629374}"


In [40]:
from surprise import BaselineOnly
from surprise import accuracy
data = Dataset.load_builtin('ml-100k')
data.split(n_folds=3)

algo = BaselineOnly()

for trainset, testset in data.folds():

    # train and test algorithm.
    algo.train(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
RMSE: 0.9430
Estimating biases using als...
RMSE: 0.9468
Estimating biases using als...
RMSE: 0.9506
